In [121]:
# Download data fro3m http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
# %lprun -f

In [139]:
import numpy as np
import re
import collections
import operator
from scipy.optimize import minimize
import copy
import cvxopt

from scipy.optimize import fmin_bfgs

In [140]:
def findDic(text):
    words = re.compile('[a-zA-Z\']*[a-zA-Z]').findall(text)
    return collections.Counter(words)

In [141]:
def readData(directory, paths, limit=None):
    if limit:
        paths = paths[0:limit]

    data = []
    for path in paths:
        txt = open(directory + path).read()
        data.append(txt)
    return data
        
        
negDataPath = !ls data/neg/
negData = readData('./data/neg/', negDataPath, 1000)

posDataPath = !ls data/pos/
posData = readData('./data/pos/', posDataPath, 1000)

In [142]:
# negData = [
#     'I am exhausted of this work.',
#     "I can't cooperate with this",
#     'I do not enjoy my job',
#     "I feel brilliant!",
#     'Gary is a friend of mine.',
#     'The date was good.'
# ]
# posData = [
#     'I love this burger.',
#     'This is an brilliant place!',
#     'I feel very good about these dates.',
#     'This is my best work.',
#     "What an awesome view"
# ]

# test = [
#     "I am not feeling well today.",
#     "I can't believe I'm doing this.",
#     'The date was good.',
#     "I am not feeling well today.",
#     "I can't believe I'm doing this.",
#     'I do not enjoy my job'
# ]

In [143]:
print negData[0]
print "+++++"
print posData[0]

Story of a man who has unnatural feelings for a pig. Starts out with a opening scene that is a terrific example of absurd comedy. A formal orchestra audience is turned into an insane, violent mob by the crazy chantings of it's singers. Unfortunately it stays absurd the WHOLE time with no general narrative eventually making it just too off putting. Even those from the era should be turned off. The cryptic dialogue would make Shakespeare seem easy to a third grader. On a technical level it's better than you might think with some good cinematography by future great Vilmos Zsigmond. Future stars Sally Kirkland and Frederic Forrest can be seen briefly.
+++++
Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High's satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right throug

In [144]:
def findWordCount(data):
    wordCount = []
    for review in data:
        wordCount.append(findDic(review))
    return wordCount

negDataDic = findWordCount(negData)
posDataDic = findWordCount(posData)

In [145]:
def updateDic(mainDic, dicArray):
    mainDic = copy.deepcopy(mainDic)
    for dic in dicArray:
        mainDic.update(dic)
    
    # update indexes
    for index, key in enumerate(mainDic):
        mainDic[ key ] = index
        
    return mainDic

allWords = {}
allWords = updateDic(allWords, negDataDic)
allWords = updateDic(allWords, posDataDic)

In [146]:
def findRow(words, text, boolean=False):
    row = np.zeros(len(allWords))
    wordsDic = findDic(text)
    for word in words:
        if word in allWords:
            row[allWords[word]] = 1 if boolean else wordsDic[word]
    return row

In [147]:
def createTable(allWords, negData, y0, posData, y1):
    x = []
    for data in negData + posData:
        x.append( findRow(allWords, data) )
    x = np.row_stack(x)
    
    y = np.append(y0 * np.ones(len(negData)), y1 * np.ones(len(posData)))
    return x, y

In [148]:
x, y = createTable(allWords, negData, -100, posData, 100)

In [150]:
M = len(x)
k = np.zeros( (M, M) )

In [151]:
for i in xrange(M):
    for j in xrange(M):
        k[i][j] = np.dot(x[i].T, x[j])

In [152]:
P = cvxopt.matrix(np.outer(y, y) * k)
q = cvxopt.matrix(np.ones(M) * -1)

G = cvxopt.matrix(np.vstack([
    np.eye(M) * -1,
    np.eye(M)
    ]))

h = cvxopt.matrix(np.hstack([
    np.zeros(M),
    np.ones(M) * 999999999.0
    ]))

A = cvxopt.matrix(y, (1,M))
b = cvxopt.matrix(0.0)

solution = cvxopt.solvers.qp(P, q, G, h, A, b)

     pcost       dcost       gap    pres   dres
 0:  9.9023e+14 -1.2087e+16  2e+16  6e-06  2e-03
 1:  5.1801e+14 -4.3733e+15  7e+15  2e-06  2e-03
 2:  2.6731e+14 -1.5819e+15  2e+15  3e-07  1e-03
 3:  1.0066e+14 -3.9671e+14  6e+14  5e-08  1e-03
 4:  1.8342e+13 -2.1509e+13  4e+13  9e-09  6e-04
 5:  2.6415e+12 -2.8861e+12  6e+12  2e-09  3e-04
 6:  3.7857e+11 -4.1627e+11  8e+11  3e-09  1e-04
 7:  5.4247e+10 -5.9615e+10  1e+11  9e-10  4e-05
 8:  7.7735e+09 -8.5431e+09  2e+10  4e-10  1e-05
 9:  1.1139e+09 -1.2242e+09  2e+09  2e-10  6e-06
10:  1.5962e+08 -1.7543e+08  3e+08  2e-10  2e-06
11:  2.2874e+07 -2.5138e+07  5e+07  5e-11  7e-07
12:  3.2777e+06 -3.6023e+06  7e+06  7e-12  3e-07
13:  4.6968e+05 -5.1621e+05  1e+06  8e-12  1e-07
14:  6.7301e+04 -7.3975e+04  1e+05  7e-13  4e-08
15:  9.6428e+03 -1.0602e+04  2e+04  3e-13  2e-08
16:  1.3813e+03 -1.5197e+03  3e+03  2e-13  6e-09
17:  1.9775e+02 -2.1796e+02  4e+02  2e-14  2e-09
18:  2.8267e+01 -3.1303e+01  6e+01  2e-14  1e-09
19:  4.0238e+00 -4.51

In [153]:
solution

{'dual infeasibility': 2.036025600692497e-12,
 'dual objective': -0.0006956387145998656,
 'dual slack': 1.6732995466410092e-21,
 'gap': 1.4443135230886341e-08,
 'iterations': 29,
 'primal infeasibility': 2.1763410436867817e-16,
 'primal objective': -0.0006956242714646348,
 'primal slack': 3.0927131461344753e-13,
 'relative gap': 2.0762839687114947e-05,
 's': <4000x1 matrix, tc='d'>,
 'status': 'optimal',
 'x': <2000x1 matrix, tc='d'>,
 'y': <1x1 matrix, tc='d'>,
 'z': <4000x1 matrix, tc='d'>}

In [154]:
findRow(allWords, test[0])

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [155]:
a

array([  2.04420607e-05,   2.52993250e-05,   1.68320264e-05,
         6.16963768e-05,   1.87029236e-05,   3.04491323e-05,
         6.20560736e-05,   2.50502122e-05,   3.87621427e-05,
         2.45711919e-05,   2.29822245e-05])

In [156]:
w = np.zeros(len(x[0]))

In [157]:
for i in xrange(len(a)):
    w += a[i] * y[i] * x[i]

In [158]:
w

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [159]:
w.dot( findRow(allWords, negData[1]) )

-46.513057389909314

In [160]:
negMax = None
for i in negData:
    t = w.dot( findRow(allWords, negData[1]) )
    if not negMax or t > negMax:
        negMax = t

In [161]:
posMin = None
for i in posData:
    t = w.dot( findRow(allWords, posData[1]) )
    if not posMin or t < posMin:
        posMin = t

In [182]:
b = (negMax+posMin)/2

In [187]:
w.dot( findRow(allWords, posData[3]) ) + b

-43.036562345676963